WEB SCRAPPING TRANSFERMARKT

In [10]:
import lxml
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [15]:
# URL de la página de Mbappé en Transfermarkt
url = 'https://www.transfermarkt.es/kylian-mbappe/profil/spieler/342229'

# Hacemos la solicitud con un user-agent válido para no ser bloqueados
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')


In [19]:
# Extraer el nombre completo
try:
    headline_wrapper = soup.find('h1', class_='data-header__headline-wrapper')
    numero = headline_wrapper.find('span', class_='data-header__shirt-number').text.strip()
    nombre = headline_wrapper.text.replace(numero, "").strip()
except AttributeError:
    nombre = "Nombre no encontrado"

# Imprimir el nombre del jugador
print(f"Nombre: {nombre}")

Nombre: Kylian Mbappé


In [20]:
# Extraer la información del fichaje
try:
    club_info = soup.find('div', class_='data-header__club-info')
    
    # Nombre del equipo
    equipo = club_info.find('span', class_='data-header__club').find('a').text.strip()
    
    # Liga
    liga = club_info.find('span', class_='data-header__league').text.strip()
    
    # Fecha de fichaje
    fecha_fichaje = club_info.find_all('span', class_='data-header__label')[1].find('span', class_='data-header__content').text.strip()
    
    # Contrato hasta
    contrato_hasta = club_info.find_all('span', class_='data-header__label')[2].find('span', class_='data-header__content').text.strip()

except AttributeError:
    equipo = liga = fecha_fichaje = contrato_hasta = "Información no encontrada"

# Imprimir la información del fichaje
print(f"Equipo: {equipo}")
print(f"Liga: {liga}")
print(f"Fecha de Fichaje: {fecha_fichaje}")
print(f"Contrato hasta: {contrato_hasta}")

Equipo: Real Madrid
Liga: LaLiga
Fecha de Fichaje: 01/07/2024
Contrato hasta: 30/06/2029


In [23]:
# Extraer la fecha de nacimiento y edad
try:
    birth_date_span = soup.find('span', {'itemprop': 'birthDate'})
    if birth_date_span:
        fecha_nacimiento = birth_date_span.get_text(strip=True)
    else:
        fecha_nacimiento = "Fecha de nacimiento no encontrada"
except AttributeError:
    fecha_nacimiento = "Fecha de nacimiento no encontrada"

# Imprimir la fecha de nacimiento
print(f"Fecha de Nacimiento: {fecha_nacimiento}")

Fecha de Nacimiento: 20/12/1998 (25)


In [31]:
# Extraer el lugar de nacimiento
try:
    birth_place_span = soup.find('span', {'itemprop': 'birthPlace'})
    if birth_place_span:
        lugar_nacimiento = birth_place_span.get_text(strip=True)
    else:
        lugar_nacimiento = "Lugar de nacimiento no encontrado"
except AttributeError:
    lugar_nacimiento = "Lugar de nacimiento no encontrado"

print(f"Lugar de Nacimiento: {lugar_nacimiento}")

Lugar de Nacimiento: Paris


In [32]:
# Extraer la nacionalidad
try:
    nationality_span = soup.find('span', {'itemprop': 'nationality'})
    if nationality_span:
        nacionalidad = nationality_span.get_text(strip=True).split('\n')[-1].strip()
    else:
        nacionalidad = "Nacionalidad no encontrada"
except AttributeError:
    nacionalidad = "Nacionalidad no encontrada"

print(f"Nacionalidad: {nacionalidad}")


Nacionalidad: Francia


In [33]:
# Extraer la altura
try:
    height_span = soup.find('span', {'itemprop': 'height'})
    if height_span:
        altura = height_span.get_text(strip=True)
    else:
        altura = "Altura no encontrada"
except AttributeError:
    altura = "Altura no encontrada"

print(f"Altura: {altura}")

Altura: 1,78 m


In [47]:
# Extraer la posición
try:
    # Encontrar todos los <ul> con la clase 'data-header__items'
    uls = soup.find_all('ul', class_='data-header__items')
    if len(uls) > 1:
        # Dentro del segundo <ul>, buscar el <li> que contiene 'Posición:'
        position_li = next((li for li in uls[1].find_all('li', class_='data-header__label') 
                            if 'Posición:' in li.get_text()), None)
        if position_li:
            position_span = position_li.find('span', class_='data-header__content')
            #posicion = get_text_or_default(position_span)
        else:
            posicion = "Posición no encontrada"
    else:
        posicion = "Posición no encontrada"
except AttributeError:
    posicion = "Posición no encontrada"

# Imprimir la posición
print(f"Posición: {posicion}")

Posición: Delantero centro


In [53]:
# Extraer la selección
try:
    # Encontrar el <li> que contiene 'Selección:'
    selection_li = next((li for li in soup.find_all('li', class_='data-header__label') 
                         if 'Selección:' in li.get_text()), None)
    if selection_li:
        selection_span = selection_li.find('span', class_='data-header__content')
        if selection_span:
            # Extraer y limpiar el texto de la selección
            selection = selection_span.get_text(strip=True)
            # Limpiar el texto de posibles elementos adicionales (imágenes y enlaces)
            selection = ' '.join(selection.split())
        else:
            selection = "Selección no encontrada"
    else:
        selection = "Selección no encontrada"
except AttributeError:
    selection = "Selección no encontrada"

# Imprimir la selección
print(f"Selección: {selection}")

Selección: Francia


In [50]:
# Extraer partidos internacionales y goles
try:
    # Encontrar el <li> que contiene 'Partidos internac./goles:'
    stats_li = next((li for li in soup.find_all('li', class_='data-header__label') 
                     if 'Partidos internac./goles:' in li.get_text()), None)
    if stats_li:
        # Encontrar todos los <a> dentro del <li> correspondiente
        stats_links = stats_li.find_all('a', class_='data-header__content--highlight')
        if len(stats_links) == 2:
            partidos = stats_links[0].get_text(strip=True)
            goles = stats_links[1].get_text(strip=True)
        else:
            partidos = "No disponible"
            goles = "No disponible"
    else:
        partidos = goles = "Partidos y goles no encontrados"
except AttributeError:
    partidos = goles = "Partidos y goles no encontrados"

print(f"Partidos Internacionales: {partidos}")
print(f"Goles: {goles}")

Partidos Internacionales: 84
Goles: 48


In [52]:
# Extraer el valor del mercado y la última revisión
try:
    # Buscar el <div> que contiene el valor del mercado
    market_value_div = soup.find('div', class_='data-header__box--small')
    if market_value_div:
        # Extraer el valor del mercado
        market_value_span = market_value_div.find('span', class_='waehrung')
        market_value_text = market_value_div.get_text(separator=' ', strip=True)
        
        # Limpiar el valor del mercado de la última revisión
        if market_value_span:
            market_value = market_value_text.split('Última revisión:')[0].strip()
        else:
            market_value = market_value_text.split('Última revisión:')[0].strip()
        
        # Extraer la última revisión
        last_update = market_value_text.split('Última revisión:')[-1].strip() if 'Última revisión:' in market_value_text else "Última revisión no encontrada"
    else:
        market_value = "Valor de mercado no encontrado"
        last_update = "Última revisión no encontrada"
except AttributeError:
    market_value = "Valor de mercado no encontrado"
    last_update = "Última revisión no encontrada"

# Imprimir la información extraída
print(f"Valor de Mercado: {market_value}")
print(f"Última Revisión: {last_update}")

Valor de Mercado: 180,00 mill. €
Última Revisión: 03/06/2024
